In [5]:
import cv2
import numpy as np

# Load the images
background = cv2.imread('1.jpeg')
foreground = cv2.imread('2.png')

# Get the dimensions of both images
bg_height, bg_width = background.shape[:2]
fg_height, fg_width = foreground.shape[:2]

# Create a mask for the foreground (white where the object is, black otherwise)
gray_foreground = cv2.cvtColor(foreground, cv2.COLOR_BGR2GRAY)
_, mask = cv2.threshold(gray_foreground, 1, 255, cv2.THRESH_BINARY)

# Ensure that the foreground fits within the background, if not resize foreground
if fg_width > bg_width or fg_height > bg_height:
    scale_w = bg_width / fg_width
    scale_h = bg_height / fg_height
    scale = min(scale_w, scale_h)  # Scale to fit within background
    foreground = cv2.resize(foreground, (int(fg_width * scale), int(fg_height * scale)))
    mask = cv2.resize(mask, (int(fg_width * scale), int(fg_height * scale)))
    fg_height, fg_width = foreground.shape[:2]  # Update dimensions after resizing

# Calculate the center of the background ensuring the foreground fits
center_x = bg_width // 2
center_y = bg_height // 2
center = (center_x, center_y)

# Check if the ROI for the foreground exceeds the background boundaries
if center_x - fg_width // 2 < 0 or center_x + fg_width // 2 > bg_width or \
   center_y - fg_height // 2 < 0 or center_y + fg_height // 2 > bg_height:
    print("Error: Foreground exceeds background bounds")
else:
    # Perform Poisson blending
    blended_image = cv2.seamlessClone(foreground, background, mask, center, cv2.NORMAL_CLONE)

    # Display and save the result
    cv2.imshow('Blended Image', blended_image)
    cv2.imwrite('blended_output_fixed.jpg', blended_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [19]:
import cv2
import numpy as np

# Load the foreground (car) and background (street) images
foreground = cv2.imread('car.png', cv2.IMREAD_UNCHANGED)
background = cv2.imread('street.jpg')

# If the foreground is RGBA (with alpha), separate the alpha channel
if foreground.shape[2] == 4:
    alpha_channel = foreground[:, :, 3]  # Get the alpha channel
    foreground = foreground[:, :, :3]    # Remove the alpha channel for blending

# Create a mask from the alpha channel (if available), otherwise create a binary mask
mask = cv2.cvtColor(foreground, cv2.COLOR_BGR2GRAY)
_, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

# Resize the car if it's too large for the street background
bg_height, bg_width = background.shape[:2]
fg_height, fg_width = foreground.shape[:2]

# Ensure the foreground fits within the background
if fg_width > bg_width or fg_height > bg_height:
    scale_factor = min(bg_width / fg_width, bg_height / fg_height)
    foreground = cv2.resize(foreground, (int(fg_width * scale_factor), int(fg_height * scale_factor)))
    mask = cv2.resize(mask, (int(fg_width * scale_factor), int(fg_height * scale_factor)))

# Define the position where the car should be placed on the street
# Let's position it in the middle of the street
center_x = bg_width // 2
center_y = int(bg_height * 0.75)  # Lower part of the street
center = (center_x, center_y)

# Perform Poisson blending
blended_image = cv2.seamlessClone(foreground, background, mask, center, cv2.NORMAL_CLONE)

# Show the result
cv2.imshow('Blended Image', blended_image)
cv2.imwrite('blended_car_on_street.jpg', blended_image)
cv2.waitKey(0)
cv2.destroyAllWindows()